In [181]:
from pathlib import Path
import json

from pptx2md.parser import is_title, is_list_block, is_strong, is_accent
import pptx

In [230]:
DATA_DIR = Path('repo/data')
DATA_DIR

PosixPath('repo/data')

In [231]:
example_slide_path = DATA_DIR / '1_raw' / 'Aula02 - Teoria - 2021.pptx'
example_slide_path

PosixPath('repo/data/1_raw/Aula02 - Teoria - 2021.pptx')

In [232]:
example_presentation = pptx.Presentation(example_slide_path)
example_presentation

In [154]:
example_presentation.slides

In [155]:
text_runs = {}

for i, slide in enumerate(example_presentation.slides):
    text_runs[i] = []
    for shape in slide.shapes:
        if not shape.has_text_frame:
            continue
        for paragraph in shape.text_frame.paragraphs:
            for run in paragraph.runs:
                text_runs[i].append(run.text)

text_runs

{0: ['Programação de Sistemas',
  '(Sistemas de Programação)',
  'Máquinas de Turing',
  'PCS3616'],
 1: ['Roteiro desta Aula',
  'Introdução às Máquinas de Turing',
  'Introdução',
  'Ações básicas de uma Máquina de Turing',
  'Definição formal de uma Máquina de Turing',
  'Linguagens decidíveis',
  'Aplicações das Máquinas de Turing',
  'Computação de funções',
  'Uma Máquina de Turing para Somar em Unário'],
 2: ['Alan Mathison Turing ',
  '(23/06/1912 – 07/06/1954)',
  '\tM',
  'atemático, lógico, criptógrafo e herói de guerra britânico. Considerado o pai da ciência da computação. Fez previsões acerca da Inteligência Artificial e propôs o Teste de Turing, contribuindo para o debate sobre a consciência das máquinas e suas capacidades de pensar. Formalizou o conceito de algoritmo e computação com a Máquina de Turing, gerando sua versão da Tese de Church-Turing. Responsável pela quebra do código alemão ',
  'Enigma',
  ' durante a II Guerra Mundial. Depois da guerra, projetou um pione

In [156]:
with open(DATA_DIR / 'test.json', 'w', encoding='utf8') as f:
    json.dump(text_runs, f)

In [157]:
for i, slide in enumerate(example_presentation.slides):
    print('Slide', i)
    for shape in slide.placeholders:
        print(shape.placeholder_format.idx, shape.name)
    print('\n')

Slide 0
0 Google Shape;51;p1
1 Google Shape;52;p1
2 Google Shape;53;p1


Slide 1
0 Google Shape;59;p2
1 Google Shape;60;p2


Slide 2
0 Google Shape;66;p3
1 Google Shape;67;p3
2 Google Shape;68;p3


Slide 3
0 Google Shape;75;p4
1 Google Shape;74;p4


Slide 4
0 Google Shape;82;p5
1 Google Shape;81;p5


Slide 5
0 Google Shape;89;p6
1 Google Shape;88;p6


Slide 6
0 Google Shape;111;p7
1 Google Shape;95;p7


Slide 7
0 Google Shape;118;p8
1 Google Shape;117;p8


Slide 8
0 Google Shape;124;p9
1 Google Shape;125;p9


Slide 9
0 Google Shape;167;p10
1 Google Shape;131;p10


Slide 10
0 Google Shape;200;p11
1 Google Shape;173;p11


Slide 11
0 Google Shape;256;p12


Slide 12
0 Google Shape;297;p13


Slide 13
0 Google Shape;317;p14
1 Google Shape;318;p14


Slide 14
0 Google Shape;336;p15
1 Google Shape;337;p15


Slide 15
0 Google Shape;343;p16
1 Google Shape;344;p16


Slide 16
0 Google Shape;351;p17
1 Google Shape;352;p17


Slide 17
0 Google Shape;369;p18
1 Google Shape;370;p18


Slide 18
0 Google S

In [158]:
example_slide = example_presentation.slides[2]

In [170]:
[shape.text for shape in example_slide.shapes if hasattr(shape, 'text')]

['Alan Mathison Turing \x0b(23/06/1912 – 07/06/1954)',
 '\tMatemático, lógico, criptógrafo e herói de guerra britânico. Considerado o pai da ciência da computação. Fez previsões acerca da Inteligência Artificial e propôs o Teste de Turing, contribuindo para o debate sobre a consciência das máquinas e suas capacidades de pensar. Formalizou o conceito de algoritmo e computação com a Máquina de Turing, gerando sua versão da Tese de Church-Turing. Responsável pela quebra do código alemão Enigma durante a II Guerra Mundial. Depois da guerra, projetou um pioneiro computador digital programável eletronicamente. Foi processado e condenado por ser homossexual (em 1952). Morreu envenenado (provável suicídio). O Turing Award foi criado em sua homenagem.']

In [176]:
[shape.image for shape in example_slide.shapes if hasattr(shape, 'image')]

In [164]:
[is_title(shape) for shape in example_slide.shapes]

[True, False, False]

In [184]:
[is_list_block(shape) for shape in example_slide.shapes if hasattr(shape, 'text_frame')]

[False, False]

In [185]:
example_shape = example_slide.shapes[0]

In [186]:
[paragraph.text for paragraph in example_shape.text_frame.paragraphs]

['Alan Mathison Turing \x0b(23/06/1912 – 07/06/1954)']

In [187]:
[(is_strong(run.font), is_accent(run.font)) for run in example_shape.text_frame.paragraphs[0].runs]

[(False, False), (False, False)]

In [188]:
[paragraph.level for paragraph in example_slide.shapes[0].text_frame.paragraphs]

[0]

In [274]:
from dataclasses import dataclass, field
from hashlib import sha1
from typing import ClassVar, Union


@dataclass
class Renderable:
    save_dirpath: Path = field(init=False, repr=False)

    def to_markdown(self) -> str:
        raise NotImplementedError


class Parser:
    @staticmethod
    def from_pptx(obj):
        raise NotImplementedError


@dataclass
class Title(Parser, Renderable):
    text: str

    def to_markdown(self) -> str:
        return self.text

    @staticmethod
    def from_pptx(obj) -> 'Title':
        return Title('## ' + obj.text.strip())


@dataclass
class BulletPoint:
    level: int
    text: str


@dataclass
class BulletPoints(Renderable):
    bullets: list[BulletPoint]

    def to_markdown(self) -> str:
        result = [
            bullet_point.level * '\t'
            + '* ' + bullet_point.text
            for bullet_point in self.bullets
        ]
        return '\n'.join(result)


@dataclass
class Paragraphs(Renderable):
    paragraphs: list[str]
    
    def to_markdown(self) -> str:
        return '\n'.join(self.paragraphs)


@dataclass
class Text(Parser, Renderable):
    is_bullet: bool
    content: Union[Paragraphs, BulletPoints]

    def to_markdown(self) -> str:
        self.content.to_markdown()

    @staticmethod
    def from_pptx(obj) -> 'Text':
        paragraphs = obj.text_frame.paragraphs
        paragraphs_text = [
            Text._parse_runs(paragraph.runs)
            for paragraph in paragraphs
        ]
        paragraphs_level = [
            paragraph.level
            for paragraph in paragraphs
        ]
        if len(set(paragraphs_level)) > 1 or len(paragraphs) > 2:
            content = BulletPoints([
                BulletPoint(level, text)
                for level, text in zip(paragraphs_level, paragraphs_text)
            ])
        else:
            content = Paragraphs(paragraphs_text)

        return content


    @staticmethod
    def _parse_runs(runs: list) -> str:
        result = []
        for run in runs:
            emphasis_markers = (
                '_'
                * (is_accent(run.font) + is_strong(run.font))
            )
            result.append(emphasis_markers + run.text.strip() + emphasis_markers)
        return ' '.join(result)


@dataclass
class Image(Parser, Renderable):
    filename: str
    image: bytes = field(repr=False)
    IMAGE_DIR: ClassVar[str] = 'img'

    def to_markdown(self) -> str:
        relative_image_filepath = Path(
            self.IMAGE_DIR,
            self.filename
        )
        image_filepath = self.save_dirpath / relative_image_filepath
        image_filepath.parent.mkdir(parents=True, exist_ok=True)
        with open(image_filepath, 'wb') as f:
            f.write(self.image)
        return f'![]({relative_image_filepath})'

    @staticmethod
    def from_pptx(obj) -> 'Image':
        image_bytes = obj.image.blob
        image_hash = sha1()
        image_hash.update(image_bytes)
        image_filename = (
            Path(obj.image.filename).stem
            + '-'
            + image_hash.hexdigest()
            + '.'+obj.image.ext
        )
        return Image(
            filename=image_filename,
            image=image_bytes,
        ) 


@dataclass
class Slide(Parser):
    title: Title
    content: list[Renderable] = field(repr=False)

    def save(self, output_path: Path) -> str:
        result = [self.title.to_markdown()]
        for renderable in self.content:
            renderable.save_dirpath = output_path
            result.append(renderable.to_markdown())
        return '\n\n'.join(result)

    @staticmethod
    def from_pptx(obj) -> 'Slide':
        content = []
        title = Title('Title placeholder')
        for shape in obj.shapes:
            if hasattr(shape, 'image'):
                content.append(Image.from_pptx(shape))
            elif is_title(shape):
                title = Title.from_pptx(shape)
            elif hasattr(shape, 'text_frame'):
                content.append(Text.from_pptx(shape))
        
        return Slide(
            title=title,
            content=content,
        )


@dataclass
class Presentation(Parser):
    slides: list[Slide] = field(repr=False)
    OUTPUT_FILENAME: ClassVar[str] = 'out.md'

    def save(self, output_path: Path) -> None:
        result = []
        for slide in self.slides:
            result.append(slide.save(output_path))
        with open(output_path / self.OUTPUT_FILENAME, 'w', encoding='utf8') as f:
            f.write('\n\n\n'.join(result))

    @staticmethod
    def read(path: Path) -> 'Presentation':
        pptx_presentation = pptx.Presentation(path)
        return Presentation.from_pptx(pptx_presentation)

    @staticmethod
    def from_pptx(obj) -> 'Presentation':
        return Presentation([
            Slide.from_pptx(slide)
            for slide in obj.slides
        ])


In [275]:
example_custom_presentation = Presentation.from_pptx(example_presentation)
example_custom_presentation

Presentation()

In [276]:
example_custom_presentation.slides[2].content

[Paragraphs(paragraphs=['_M_ atemático, lógico, criptógrafo e herói de guerra britânico. Considerado o pai da ciência da computação. Fez previsões acerca da Inteligência Artificial e propôs o Teste de Turing, contribuindo para o debate sobre a consciência das máquinas e suas capacidades de pensar. Formalizou o conceito de algoritmo e computação com a Máquina de Turing, gerando sua versão da Tese de Church-Turing. Responsável pela quebra do código alemão _Enigma_ durante a II Guerra Mundial. Depois da guerra, projetou um pioneiro computador digital programável eletronicamente. Foi processado e condenado por ser homossexual (em 1952). Morreu envenenado (provável suicídio). O Turing Award foi criado em sua homenagem.']),
 Image(filename='image-027444f8bae0e864345c78b90f2ef2e050d0f96c.jpg')]

In [277]:
example_custom_presentation.save(DATA_DIR / 'test')